<a href="https://colab.research.google.com/github/maninog/langchain/blob/main/LangChain_Memory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangChain Memory

## README
- author: [Masumi Morishige](https://twitter.com/masumi_creator)
- created_at: 2023-05-02
- updated_at: 2023-05-06

### 実行方法
1. OpenAIのAPIキーを発行
2. `os.environ["OPENAI_API_KEY"] = "..."`の`""`の中にご自身のAPIキーを代入
3. 「ランタイム > すべてのセルを実行」を実行

### 参考情報
- Zenn: [LangChain Memoryとは？【Chat Message History・Conversation Buffer Memory】](https://zenn.dev/umi_mori/books/prompt-engineer/viewer/langchain_memory)
- YouTube: [LangChain Memoryとは？【Chat Message History / Conversation Buffer Memory】](https://youtu.be/57egG_y4vUE)

### OpenAI APIの発行方法

[<img src="https://img.youtube.com/vi/frpsKLNW1q4/maxresdefault.jpg" width="600px">](https://youtu.be/frpsKLNW1q4)

[【エンジニア向け】OpenAIのAPI連携方法【環境構築 + GASによるGoogle Documentへの組み込み】](https://youtu.be/frpsKLNW1q4)

## 環境構築

In [ ]:
!pip3 install langchain openai

In [ ]:
import os

#TODO: APIキーの登録が必要
os.environ["OPENAI_API_KEY"] = "..."

## 1. Chat Message Historyの使い方

In [ ]:
from langchain.memory import ChatMessageHistory
from langchain.schema import messages_to_dict
from langchain.schema import messages_from_dict

history = ChatMessageHistory()

history.add_user_message("AIとは何？")
history.add_ai_message("AIとは、人工知能のことです。")

message_list = history.messages
message_dict = messages_to_dict(message_list)
print(message_list)
print(message_dict)

message_list_from_dict = messages_from_dict(message_dict)
print(message_list_from_dict)

## 2. Simple Memoryの使い方

In [ ]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.memory import SimpleMemory

llm = OpenAI(model_name="text-davinci-003")
prompt_1 = PromptTemplate(
    input_variables=["adjective", "job", "time"],
    template="{time}において、{adjective}{job}に一番オススメのプログラミング言語は?\nプログラミング言語：",
)
chain_1 = LLMChain(llm=llm, prompt=prompt_1, output_key="programming_language")

prompt_2 = PromptTemplate(
    input_variables=["programming_language", "time"],
    template="現在は{time}だとします。{programming_language}を学ぶためにやるべきことを3ステップで100文字で教えて。",
)
chain_2 = LLMChain(llm=llm, prompt=prompt_2, output_key="learning_step")

overall_chain = SequentialChain(
    chains=[chain_1, chain_2],
    input_variables=["adjective", "job"],
    output_variables=["programming_language", "learning_step"],
    verbose=True,
    memory=SimpleMemory(memories={"time": "2030年"}),
)
output = overall_chain({
    "adjective": "ベテランの",
    "job": "エンジニア",
})
print(output)

## 3. Buffer Memoryの使い方

In [ ]:
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

memory.save_context(
    {"input": "AIとは何？"},
    {"output": "AIとは、人工知能のことです。"},
)
print(memory.load_memory_variables({}))

In [ ]:
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)

memory.save_context(
    {"input": "AIとは何？"},
    {"output": "AIとは、人工知能のことです。"},
)
print(memory.load_memory_variables({}))

In [ ]:
from langchain.llms import OpenAI
from langchain.chains import ConversationChain


llm = OpenAI(model_name="text-davinci-003")
conversation = ConversationChain(
    llm=llm,
    verbose=True,
    memory=ConversationBufferMemory()
)

conversation("AIとは何？")

In [ ]:
conversation("より詳しく教えて。")

## 4. LLMChainでの実装方法

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain import OpenAI, LLMChain, PromptTemplate

template = """あなたは人間と話すチャットボットです。

{chat_history}
Human: {human_input}
Chatbot:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input"],
    template=template
)
memory = ConversationBufferMemory(memory_key="chat_history")

llm_chain = LLMChain(
    llm=OpenAI(model_name="text-davinci-003"),
    prompt=prompt,
    verbose=True,
    memory=memory,
)

llm_chain.predict(human_input="AIとは何？")

In [ ]:
llm_chain.predict(human_input="より詳しく教えて。")